![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [23]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# Start coding here...

In [5]:
df = pd.read_csv("bank_marketing.csv")
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [4]:
#Exploring the Shape of Data, Info & summary stats
df.shape

(41188, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [7]:
df.describe()

,client_id,age,day,contact_duration,number_contacts,previous_campaign_contacts,cons_price_idx,euribor_three_months
count,41188.00000,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,20593.50000,40.02406,15.730334,258.285010,2.567593,0.172963,93.575664,3.621291
std,11890.09578,10.42125,8.889867,259.279249,2.770014,0.494901,0.578840,1.734447
min,0.00000,17.00000,1.000000,0.000000,1.000000,0.000000,92.201000,0.634000
25%,10296.75000,32.00000,8.000000,102.000000,1.000000,0.000000,93.075000,1.344000
50%,20593.50000,38.00000,16.000000,180.000000,2.000000,0.000000,93.749000,4.857000
75%,30890.25000,47.00000,23.000000,319.000000,3.000000,0.000000,93.994000,4.961000
max,41187.00000,98.00000,31.000000,4918.000000,56.000000,7.000000,94.767000,5.045000


In [88]:
# the Data frame contains unique rows for every customer/client
df["client_id"].nunique()

41188

In [32]:
#splitting Data into 3 Data Frameworks
client   =df[["client_id","age", "job", "marital","education","credit_default", "mortgage"]]


In [61]:
campaign =df[["client_id","number_contacts", "contact_duration", "previous_campaign_contacts",
              "previous_outcome","campaign_outcome", "month", "day"]]


In [ ]:
economics = df[["client_id", "cons_price_idx","euribor_three_months"]]

In [33]:
# Cleaning client Dataframe
client["job"].unique()

array(['housemaid', 'services', 'admin.', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [34]:
# Cleaning client Dataframe
client["education"].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

In [35]:
# Client's type of job > Change "." to "_" &  
# Client's level of education Change "." to "_" and "unknown" to np.NaN
client["job"] = client["job"].str.replace(".","_")
client["education"] = client["education"].str.replace(".","_")
client["education"] = client["education"].replace("unknown", np.nan)

In [36]:
client["job"].unique()

array(['housemaid', 'services', 'admin_', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [37]:
client["education"].unique()

array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [38]:
client["credit_default"].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [39]:
client["mortgage"].unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [40]:
#Whether the client's credit is in default	Convert to boolean data type:
# 1 if "yes", otherwise 0
for col in ["credit_default", "mortgage"]:
    client[col] = client[col].map({"yes" : 1 ,
                                   "no" : 0 ,
                                   "unknown" : 0 } )
    client[col] = client[col].astype(bool)

In [41]:
client["credit_default"].unique()

array([False,  True])

In [43]:
client["mortgage"].unique()

array([False,  True])

In [44]:
#Cleaning campaign DF
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


In [62]:
#previous_outcome Outcome of the previous campaign > Convert to boolean data type: 1 if "success", otherwise 0.
#campaign_outcome Outcome of the current campaign > Convert to boolean data type: 1 if "success", otherwise 0.
campaign["previous_outcome"].unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [63]:
campaign["campaign_outcome"].unique()

array(['no', 'yes'], dtype=object)

In [64]:
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success" : 1,"failure" :0, "nonexistent":0  })

In [67]:
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no":0  })

In [72]:
#add a year column to append day- month & year
campaign["year"] = "2022"

In [73]:
campaign["campaign_outcome"].unique()

array([0, 1])

In [74]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year
0,0,1,261,0,0,0,may,13,2022
1,1,1,149,0,0,0,may,19,2022
2,2,1,226,0,0,0,may,23,2022
3,3,1,151,0,0,0,may,27,2022
4,4,1,307,0,0,0,may,3,2022


In [84]:
#converting both previous_outcome & campaign_outcome to Booleans
for col in (["previous_outcome","campaign_outcome"]):
    campaign[col] = campaign[col].astype(bool)

In [76]:
# last_contact_date Create from a combination of day, month, and a newly created year column 
# (which should have a value of 2022);Format = "YYYY-MM-DD"
campaign["day"] = campaign["day"].astype(str)
campaign["last_contact_date"] = campaign["year"] + '-' + campaign["month"] + '-' + campaign["day"]


In [77]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,year,last_contact_date
0,0,1,261,0,0,0,may,13,2022,2022-may-13
1,1,1,149,0,0,0,may,19,2022,2022-may-19
2,2,1,226,0,0,0,may,23,2022,2022-may-23
3,3,1,151,0,0,0,may,27,2022,2022-may-27
4,4,1,307,0,0,0,may,3,2022,2022-may-3


In [80]:
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = '%Y-%b-%d')

In [81]:
#dropping un-necessary colummns
campaign = campaign.drop(columns = ["month", "day", "year"])

In [85]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [89]:
# saving 3 Different data frames after cleaning
client.to_csv("client.csv", index = False)
campaign.to_csv("campaign.csv", index = False)
economics.to_csv("economics.csv", index = False)